In [1]:
# Install PySpark
!pip install -q pyspark
!pip install -q findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import os
!find /usr/local -name "pyspark"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"
os.environ["JAVA_HOME"] = "/usr"

/usr/local/bin/pyspark
/usr/local/lib/python3.10/dist-packages/pyspark
/usr/local/lib/python3.10/dist-packages/pyspark/bin/pyspark
/usr/local/lib/python3.10/dist-packages/pyspark/python/pyspark


In [3]:
# Principaux import
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkConf

# pour les dataframe et udf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory")

  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")

  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1687293510764


In [4]:
import duckdb

def demarrer_duckdb():
    # Créez une connexion DuckDB
    con = duckdb.connect('mypath.db')  # Vous pouvez remplacer 'mypath.db' par le chemin vers votre fichier de base de données DuckDB.
    print("session DuckDB démarrée")
    return con
con = demarrer_duckdb()

session DuckDB démarrée


In [5]:
url = "/content/drive/MyDrive/data/tpmod6/yellow_tripdata_2023-01"
url2 = "/content/drive/MyDrive/data/tpmod6/yellow_tripdata_2023-02"
url3 = "/content/drive/MyDrive/data/tpmod6/yellow_tripdata_2023-03"
url4 = "/content/drive/MyDrive/data/tpmod6/yellow_tripdata_2022-10"
url5 = "/content/drive/MyDrive/data/tpmod6/yellow_tripdata_2022-11"
url6 = "/content/drive/MyDrive/data/tpmod6/yellow_tripdata_2022-12"

In [6]:
def make_csv_file(url_parquet, url_csv):
  # Read the data from the parquet file into a Spark DataFrame
  df_spark = spark.read.parquet(url_parquet, inferSchema=True, header=True)

  # Write the DataFrame to a Parquet file
  df_spark.write.csv(url_csv, mode='overwrite')

In [7]:
#make_csv_file(url4+'.parquet', url4+'.csv')
#make_csv_file(url5+'.parquet', url5+'.csv')
#make_csv_file(url6+'.parquet', url6+'.csv')

In [8]:
import time

def measure_time(func, *args, **kwargs):
    start = time.time()
    func(*args, **kwargs)
    end = time.time()
    return end - start

In [31]:
def work_df(df):
    #df.printSchema()
    print(df.count())

def read_csv_spark(csv_file_path):
    df = spark.read.csv(csv_file_path)
    # Any other operation you want to do with df

    work_df(df)

    return df

def read_parquet_spark(parquet_file_path):
    df = spark.read.parquet(parquet_file_path)
    # Any other operation you want to do with df

    work_df(df)

    return df

def read_csv_duckDB(csv_file_path):
    con.execute("DROP TABLE IF EXISTS csv_table")  # Add this line
    df = con.execute(f"""
        CREATE TABLE csv_table AS SELECT * FROM
        read_csv_auto('{csv_file_path}')
    """).fetchdf()

    count = con.execute("SELECT COUNT(*) FROM csv_table").fetchone()[0]
    print(f"Number of rows: {count}")

    return df

def read_parquet_duckDB(parquet_file_path):
    con.execute("DROP TABLE IF EXISTS parquet_table")  # Add this line
    df = con.execute(f"""
        CREATE TABLE parquet_table AS SELECT * FROM
        parquet_scan('{parquet_file_path}')
    """).fetchdf()

    count = con.execute("SELECT COUNT(*) FROM parquet_table").fetchone()[0]
    print(f"Number of rows: {count}")

    return df

In [32]:
# Establish a connection to DuckDB
con = duckdb.connect(':memory:')

In [33]:
paths = [['/content/drive/MyDrive/data/tpmod6/yellow_tripdata_2023-01.parquet', '/content/drive/MyDrive/data/tpmod6/yellow_tripdata_2023-01.csv/part-00000-5adedc31-8194-425d-a703-5ca102704ca4-c000.csv'],
 ['/content/drive/MyDrive/data/tpmod6/yellow_tripdata_2023-02.parquet', '/content/drive/MyDrive/data/tpmod6/yellow_tripdata_2023-02.csv/part-00000-42c55ae7-8221-4c1f-bfdc-378f325bd142-c000.csv'],
  ['/content/drive/MyDrive/data/tpmod6/yellow_tripdata_2023-03.parquet', '/content/drive/MyDrive/data/tpmod6/yellow_tripdata_2023-03.csv/part-00000-a040099b-fdd3-4ad8-aeec-2ebd3d621d92-c000.csv'],
    ['/content/drive/MyDrive/data/tpmod6/yellow_tripdata_2022-10.parquet', '/content/drive/MyDrive/data/tpmod6/yellow_tripdata_2022-10.csv/part-00000-99c0ab56-4d71-4117-b6d9-c5e7bae115fc-c000.csv'],
      ['/content/drive/MyDrive/data/tpmod6/yellow_tripdata_2022-11.parquet', '/content/drive/MyDrive/data/tpmod6/yellow_tripdata_2022-11.csv/part-00000-3e752cbb-5f4f-4647-8030-96fce5a12b16-c000.csv'],
        ['/content/drive/MyDrive/data/tpmod6/yellow_tripdata_2022-12.parquet', '/content/drive/MyDrive/data/tpmod6/yellow_tripdata_2022-12.csv/part-00000-ca2f09a7-4899-4b7a-933e-702add1baf05-c000.csv']]

In [34]:
for path in paths:
  url_parquet, url_csv = path

  csv_time = measure_time(read_csv_spark, url_csv)
  parquet_time = measure_time(read_parquet_spark, url_parquet)
  print('Spark :')
  print(f"Reading the CSV file took {csv_time} seconds.")
  print(f"Reading the Parquet file took {parquet_time} seconds.")

  csv_time = measure_time(read_csv_duckDB, url_csv)
  parquet_time = measure_time(read_parquet_duckDB, url_parquet)
  print('duckdb :')
  print(f"Reading the CSV file took {csv_time} seconds.")
  print(f"Reading the Parquet file took {parquet_time} seconds.")

3066766
3066766
Spark :
Reading the CSV file took 2.6262056827545166 seconds.
Reading the Parquet file took 0.25728940963745117 seconds.


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

Number of rows: 3066766


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

Number of rows: 3066766
duckdb :
Reading the CSV file took 6.530694484710693 seconds.
Reading the Parquet file took 5.461055755615234 seconds.
2913955
2913955
Spark :
Reading the CSV file took 2.7950241565704346 seconds.
Reading the Parquet file took 0.24889135360717773 seconds.


...

Number of rows: 3252717
duckdb :
Reading the CSV file took 8.864650011062622 seconds.
Reading the Parquet file took 2.8710789680480957 seconds.
3399549
3399549
Spark :
Reading the CSV file took 2.808051347732544 seconds.
Reading the Parquet file took 0.26839709281921387 seconds.


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

Number of rows: 3399549


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

Number of rows: 3399549
duckdb :
Reading the CSV file took 14.580622911453247 seconds.
Reading the Parquet file took 2.6905972957611084 seconds.
